## Motivation for Attention

In [1]:
import sys 
import platform
import numpy as np 
import pandas as pd 
import string
import re 
import math 
from string import digits 
from tqdm import tqdm
import sklearn as sk 
import torch
from torch import nn 
import torch.nn.functional as nnfunc 
from torch.utils.data import Dataset, DataLoader

In [2]:
# What version of Python do you have?
print(f"python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print(f"Python: {sys.version}")
print(f"Pandas: {pd.__version__}")
print(f"Scikit-Learn: {sk.__version__}")

python Platform: macOS-15.4-arm64-arm-64bit
PyTorch Version: 2.6.0
Python: 3.11.7 (main, Dec 15 2023, 12:09:56) [Clang 14.0.6 ]
Pandas: 2.2.3
Scikit-Learn: 1.6.1


In [3]:
def get_device():
    has_gpu = torch.cuda.is_available()
    has_mps = torch.backends.mps.is_built()
    print ("NVIDIA/CUDA GPU is", "available" if has_gpu else "NOT AVAILABLE")
    print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
    return torch.device('mps') if has_mps else torch.device('cuda') if has_gpu else torch.device('cpu')
device = get_device()
print(f"Target device is {device}")

NVIDIA/CUDA GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


In [4]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled = scaled.permute(1, 0, 2, 3) + mask
        scaled = scaled.permute(1, 0, 2, 3)
    attention = nnfunc.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [5]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_sequence_length):
        super().__init__()
        self.max_sequence_length = max_sequence_length
        self.d_model = d_model

    def forward(self):
        even_i = torch.arange(0, self.d_model, 2).float()
        denominator = torch.pow(10000, even_i/self.d_model)
        position = (torch.arange(self.max_sequence_length)
                          .reshape(self.max_sequence_length, 1))
        even_PE = torch.sin(position / denominator)
        odd_PE = torch.cos(position / denominator)
        stacked = torch.stack([even_PE, odd_PE], dim=2)
        PE = torch.flatten(stacked, start_dim=1, end_dim=2)
        return PE

In [6]:
class SentenceEmbedding(nn.Module):
    "For a given sentence, create an embedding"
    def __init__(self, max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN):
        super().__init__()
        self.vocab_size = len(language_to_index)
        self.max_sequence_length = max_sequence_length
        self.embedding = nn.Embedding(self.vocab_size, d_model)
        self.language_to_index = language_to_index
        self.position_encoder = PositionalEncoding(d_model, max_sequence_length)
        self.dropout = nn.Dropout(p=0.1)
        self.START_TOKEN = START_TOKEN
        self.END_TOKEN = END_TOKEN
        self.PADDING_TOKEN = PADDING_TOKEN
    
    def batch_tokenize(self, batch, start_token, end_token):
        def tokenize(sentence, start_token, end_token):
            sentence_word_indices = []
            for token in sentence.split():
                if token in self.language_to_index:
                    sentence_word_indices.append(self.language_to_index[token])
            if start_token:
                sentence_word_indices.insert(0, self.language_to_index[self.START_TOKEN])
            if end_token:
                sentence_word_indices.append(self.language_to_index[self.END_TOKEN]) 
            for _ in range(len(sentence_word_indices), self.max_sequence_length):
                sentence_word_indices.append(self.language_to_index[self.PADDING_TOKEN])
            return torch.tensor(sentence_word_indices)

        tokenized = []
        for sentence_num in range(len(batch)):
           tokenized.append(tokenize(batch[sentence_num], start_token, end_token))
        tokenized = torch.stack(tokenized)
        return tokenized.to(device)
    
    def forward(self, x, start_token, end_token): # sentence
        x = self.batch_tokenize(x, start_token, end_token)
        x = self.embedding(x)
        pos = self.position_encoder().to(device)
        x = self.dropout(x + pos)
        return x

In [7]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(d_model , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask):
        batch_size, sequence_length, d_model = x.size()
        qkv = self.qkv_layer(x)
        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        qkv = qkv.permute(0, 2, 1, 3)
        q, k, v = qkv.chunk(3, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask)
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        out = self.linear_layer(values)
        return out

In [8]:
class LayerNormalization(nn.Module):
    def __init__(self, parameters_shape, eps=1e-5):
        super().__init__()
        self.parameters_shape=parameters_shape
        self.eps=eps
        self.gamma = nn.Parameter(torch.ones(parameters_shape))
        self.beta =  nn.Parameter(torch.zeros(parameters_shape))

    def forward(self, inputs):
        dims = [-(i + 1) for i in range(len(self.parameters_shape))]
        mean = inputs.mean(dim=dims, keepdim=True)
        var = ((inputs - mean) ** 2).mean(dim=dims, keepdim=True)
        std = (var + self.eps).sqrt()
        y = (inputs - mean) / std
        out = self.gamma * y + self.beta
        return out

In [9]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, hidden, drop_prob=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.linear1 = nn.Linear(d_model, hidden)
        self.linear2 = nn.Linear(hidden, d_model)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(p=drop_prob)

    def forward(self, x):
        x = self.linear1(x)
        x = self.relu(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [10]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(EncoderLayer, self).__init__()
        self.attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)
        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

    def forward(self, x, self_attention_mask):
        residual_x = x.clone()
        x = self.attention(x, mask=self_attention_mask)
        x = self.dropout1(x)
        x = self.norm1(x + residual_x)
        residual_x = x.clone()
        x = self.ffn(x)
        x = self.dropout2(x)
        x = self.norm2(x + residual_x)
        return x
 

In [11]:
class SequentialEncoder(nn.Sequential):
    def forward(self, *inputs):
        x, self_attention_mask  = inputs
        for module in self._modules.values():
            x = module(x, self_attention_mask)
        return x

In [12]:
class Encoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialEncoder(*[EncoderLayer(d_model, ffn_hidden, num_heads, drop_prob)
                                      for _ in range(num_layers)])

    def forward(self, x, self_attention_mask, start_token, end_token):
        x = self.sentence_embedding(x, start_token, end_token)
        x = self.layers(x, self_attention_mask)
        return x

In [13]:
class MultiHeadCrossAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super().__init__()
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.kv_layer = nn.Linear(d_model , 2 * d_model)
        self.q_layer = nn.Linear(d_model , d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, y, mask):
        batch_size, sequence_length, d_model = x.size() # in practice, this is the same for both languages...so we can technically combine with normal attention
        kv = self.kv_layer(x)
        q = self.q_layer(y)
        kv = kv.reshape(batch_size, sequence_length, self.num_heads, 2 * self.head_dim)
        q = q.reshape(batch_size, sequence_length, self.num_heads, self.head_dim)
        kv = kv.permute(0, 2, 1, 3)
        q = q.permute(0, 2, 1, 3)
        k, v = kv.chunk(2, dim=-1)
        values, attention = scaled_dot_product(q, k, v, mask) # We don't need the mask for cross attention, removing in outer function!
        values = values.permute(0, 2, 1, 3).reshape(batch_size, sequence_length, d_model)
        out = self.linear_layer(values)
        return out

In [14]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, ffn_hidden, num_heads, drop_prob):
        super(DecoderLayer, self).__init__()
        self.self_attention = MultiHeadAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm1 = LayerNormalization(parameters_shape=[d_model])
        self.dropout1 = nn.Dropout(p=drop_prob)

        self.encoder_decoder_attention = MultiHeadCrossAttention(d_model=d_model, num_heads=num_heads)
        self.layer_norm2 = LayerNormalization(parameters_shape=[d_model])
        self.dropout2 = nn.Dropout(p=drop_prob)

        self.ffn = PositionwiseFeedForward(d_model=d_model, hidden=ffn_hidden, drop_prob=drop_prob)
        self.layer_norm3 = LayerNormalization(parameters_shape=[d_model])
        self.dropout3 = nn.Dropout(p=drop_prob)

    def forward(self, x, y, self_attention_mask, cross_attention_mask):
        _y = y.clone()
        y = self.self_attention(y, mask=self_attention_mask)
        y = self.dropout1(y)
        y = self.layer_norm1(y + _y)

        _y = y.clone()
        y = self.encoder_decoder_attention(x, y, mask=cross_attention_mask)
        y = self.dropout2(y)
        y = self.layer_norm2(y + _y)

        _y = y.clone()
        y = self.ffn(y)
        y = self.dropout3(y)
        y = self.layer_norm3(y + _y)
        return y

In [15]:
class SequentialDecoder(nn.Sequential):
    def forward(self, *inputs):
        x, y, self_attention_mask, cross_attention_mask = inputs
        for module in self._modules.values():
            y = module(x, y, self_attention_mask, cross_attention_mask)
        return y

In [16]:
class Decoder(nn.Module):
    def __init__(self, 
                 d_model, 
                 ffn_hidden, 
                 num_heads, 
                 drop_prob, 
                 num_layers,
                 max_sequence_length,
                 language_to_index,
                 START_TOKEN,
                 END_TOKEN, 
                 PADDING_TOKEN):
        super().__init__()
        self.sentence_embedding = SentenceEmbedding(max_sequence_length, d_model, language_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.layers = SequentialDecoder(*[DecoderLayer(d_model, ffn_hidden, num_heads, drop_prob) for _ in range(num_layers)])

    def forward(self, x, y, self_attention_mask, cross_attention_mask, start_token, end_token):
        y = self.sentence_embedding(y, start_token, end_token)
        y = self.layers(x, y, self_attention_mask, cross_attention_mask)
        return y

In [17]:
class Transformer(nn.Module):
    def __init__(self, 
                d_model, 
                ffn_hidden, 
                num_heads, 
                drop_prob, 
                num_layers,
                max_sequence_length, 
                kn_vocab_size,
                english_to_index,
                hindi_to_index,
                START_TOKEN, 
                END_TOKEN, 
                PADDING_TOKEN
                ):
        super().__init__()
        self.encoder = Encoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, english_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.decoder = Decoder(d_model, ffn_hidden, num_heads, drop_prob, num_layers, max_sequence_length, hindi_to_index, START_TOKEN, END_TOKEN, PADDING_TOKEN)
        self.linear = nn.Linear(d_model, kn_vocab_size)
        self.device = device

    def forward(self, 
                x, 
                y, 
                encoder_self_attention_mask=None, 
                decoder_self_attention_mask=None, 
                decoder_cross_attention_mask=None,
                enc_start_token=False,
                enc_end_token=False,
                dec_start_token=False, # We should make this true
                dec_end_token=False): # x, y are batch of sentences
        x = self.encoder(x, encoder_self_attention_mask, start_token=enc_start_token, end_token=enc_end_token)
        out = self.decoder(x, y, decoder_self_attention_mask, decoder_cross_attention_mask, start_token=dec_start_token, end_token=dec_end_token)
        out = self.linear(out)
        return out

In [18]:
# Define parameters
d_model = 128 # Embedding size
batch_size = 32 # batch size to be fed to model 32 hindi sentence 32 english sentence at single point of time
ffn_hidden = 2048 # Number of hidden layer
num_heads = 8 # Number of attention heads
drop_prob = 0.1 # Percentage of neuron dropout
num_layers = 8 # Number of decoder-encoder
max_sequence_length = 90 #I've checked it in exploratary analysis-> 90 is ideal
chunk_size = 25000  # Process 25,000 rows at a time
num_epochs=10 # Number of epocs
START_TOKEN = '<START>' # Start token
PADDING_TOKEN = '<PADDING>' # Padding token
END_TOKEN = '<END>' # End token

## Preparing Data

### Read hindi file clean a bit and write it down again.

In [19]:
base_path = '/Users/siddharthchaudhary/Documents/git_repos/AI-Models/Transformer_from_scratch/'

In [20]:
train_en_file = base_path + 'Dataset/vocab_and_training_files/train_en.txt'
train_hn_file = base_path + 'Dataset/vocab_and_training_files/train_hn.txt'
english_vocab_file = base_path + 'Dataset/vocab_and_training_files/english_vocab.txt'
hindi_vocab_file = base_path + 'Dataset/vocab_and_training_files/hindi_vocab.txt'

model_path = base_path + "model/transformer_v1_hindi_english.pth"

In [21]:
model_path

'/Users/siddharthchaudhary/Documents/git_repos/AI-Models/Transformer_from_scratch/model/transformer_v1_hindi_english.pth'

### Reading the hindi, english text and vocab

In [22]:
with open(train_en_file, 'r') as file:
    english_sentences = file.readlines()
english_sentences = [sentence.rstrip('\n') for sentence in english_sentences]
english_sentences[:10]

['got it',
 'i m ok',
 'come in',
 'get out',
 'go away',
 'have fun',
 'have fun',
 'have fun',
 'i forgot',
 'i forgot']

In [23]:
with open(train_hn_file, 'r') as file:
    hindi_sentences = file.readlines()
hindi_sentences = [sentence.rstrip('\n') for sentence in hindi_sentences]
hindi_sentences[: 10]

['समझे कि नहीं',
 'मैं ठीक हूँ।',
 'अंदर आ जाओ।',
 'बाहर निकल जाओ',
 'चले जाओ',
 'मज़े करना।',
 'मौज करना।',
 'मज़े करो।',
 'मैं भूल गया।',
 'मैं भूल गई।']

In [24]:
# Initialize variables to store the maximum length sentence and its word count
max_length_sentence = ""
max_word_count = 0

# Iterate through sentences
for sentence in english_sentences:
    word_count = len(sentence.split())  # Count words in the current sentence
    if len(sentence) > len(max_length_sentence):  # Compare sentence length
        max_length_sentence = sentence
        max_word_count = word_count  # Update the word count

print(f"Longest Sentence: {max_length_sentence}")
print(f"Number of Words: {max_word_count}")

Longest Sentence: there are 104 members of icc 10 full members who play official test matches 34 associate members and 60 affiliate members icc is responsible for conduct and regulation of prominent international cricket tournaments specially world cup matches it appoints umpires and referees for all accepted test matches one day internationals and twenty 20 internationals every country has a national cricket board which regulates the cricket matches played in the country cricket board selects the national team also and makes arrangements for the matches both domestic and external
Number of Words: 87


In [25]:
with open(hindi_vocab_file, 'r') as file:
    hindi_vocab = file.readlines ()
hindi_vocab = [word.rstrip('\n') for word in hindi_vocab]
print(len(hindi_vocab) )

68142


In [26]:
with open (english_vocab_file, 'r') as file:
    english_vocab= file.readlines()
english_vocab = [word.rstrip('\n') for word in english_vocab]
print(len(english_vocab))
print(english_vocab[:10])

59893
['sikandara', 'bijapur', 'rumsfeld', 'pes', 'quits', 'kasavimantri', 'yaduendra', 'buxa', 'mindonly', 'laborers']


In [27]:
hindi_vocab.insert(0, START_TOKEN)
hindi_vocab.append(PADDING_TOKEN)
hindi_vocab.append(END_TOKEN)
print(len(hindi_vocab))
print(hindi_vocab[:10])

68145
['<START>', 'अर्काट', 'यवेस', 'रेगन', 'अड़े', 'क़ैदियों', '१६८८', 'कोलर्स', 'इडिपेंडेंट', 'टाँके']


In [28]:
english_vocab.insert(0, START_TOKEN) 
english_vocab.append(PADDING_TOKEN)
english_vocab.append(END_TOKEN)
print(len(english_vocab))
print(english_vocab[:20])

59896
['<START>', 'sikandara', 'bijapur', 'rumsfeld', 'pes', 'quits', 'kasavimantri', 'yaduendra', 'buxa', 'mindonly', 'laborers', 'sakhare', 'heptanesia', 'demanded', 'discontinuous', 'bootes', 'mig', 'pratipha', 'amalgamate', 'criticise']


In [29]:
index_to_hindi = {k:v for k,v in enumerate(hindi_vocab)}
hindi_to_index = {v:k for k,v in enumerate (hindi_vocab)}
index_to_english = {k:v for k,v in enumerate (english_vocab)}
english_to_index = {v:k for k,v in enumerate(english_vocab)}

In [30]:
# Get the first 20 key-value pairs
first_20_items = list(index_to_hindi.items())[:10]
# Print keys and values together
for key, value in first_20_items:
    print(f"{key}: {value}")

0: <START>
1: अर्काट
2: यवेस
3: रेगन
4: अड़े
5: क़ैदियों
6: १६८८
7: कोलर्स
8: इडिपेंडेंट
9: टाँके


In [31]:
# Get the first 20 key-value pairs
first_20_items = list(index_to_english.items())[:10]
# Print keys and values together
for key, value in first_20_items:
    print(f"{key}: {value}")

0: <START>
1: sikandara
2: bijapur
3: rumsfeld
4: pes
5: quits
6: kasavimantri
7: yaduendra
8: buxa
9: mindonly


In [32]:
print(f"Number of sentences: {len(hindi_sentences)}")
print(f"Number of sentences: {len(english_sentences)}")

Number of sentences: 124076
Number of sentences: 124076


In [33]:
class TextDataset(Dataset):
    def __init__(self, english_sentences, hindi_sentences):
        self.english_sentences = english_sentences
        self.hindi_sentences = hindi_sentences
    
    def __len__(self) :
        return len(self.english_sentences)
    
    def __getitem__(self, idx):
        return self.english_sentences[idx], self.hindi_sentences[idx]

## Training the model Model

In [34]:
hn_vocab_size = len(hindi_vocab)

In [35]:
transformer = Transformer(
    d_model,
    ffn_hidden,
    num_heads, 
    drop_prob, 
    num_layers, 
    max_sequence_length, 
    hn_vocab_size,
    english_to_index, 
    hindi_to_index,
    START_TOKEN, 
    END_TOKEN, 
    PADDING_TOKEN)

In [36]:
criterian = nn.CrossEntropyLoss(ignore_index=hindi_to_index[PADDING_TOKEN],
                                reduction= 'none')
# When computing the loss, we are ignoring cases when the label is the padding token
for params in transformer.parameters():
    if params.dim() > 1:
        nn.init.xavier_uniform_(params)
        
optim = torch.optim.Adam(transformer.parameters(), lr=1e-4)

In [37]:
NEG_INFTY = -1e9

def create_masks(eng_batch, hn_batch):
    num_sentences = len(eng_batch)
    look_ahead_mask = torch.full([max_sequence_length, max_sequence_length] , True)
    look_ahead_mask = torch.triu(look_ahead_mask, diagonal=1)
    encoder_padding_mask = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_self_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    decoder_padding_mask_cross_attention = torch.full([num_sentences, max_sequence_length, max_sequence_length] , False)
    
    for idx in range(num_sentences):
        eng_sentence_length, hn_sentence_length = len(eng_batch[idx]), len(hn_batch[idx])
        eng_words_to_padding_mask = np.arange(eng_sentence_length + 1, max_sequence_length)
        hn_words_to_padding_mask = np.arange(hn_sentence_length + 1, max_sequence_length)
        encoder_padding_mask[idx, :, eng_words_to_padding_mask] = True
        encoder_padding_mask[idx, eng_words_to_padding_mask, :] = True
        decoder_padding_mask_self_attention[idx, :, hn_words_to_padding_mask] = True
        decoder_padding_mask_self_attention[idx, hn_words_to_padding_mask, :] = True
        decoder_padding_mask_cross_attention[idx,:, eng_words_to_padding_mask] = True
        decoder_padding_mask_cross_attention[idx, hn_words_to_padding_mask, :] = True
    
    encoder_self_attention_mask = torch.where(encoder_padding_mask, NEG_INFTY, 0)
    decoder_self_attention_mask = torch.where(look_ahead_mask + decoder_padding_mask_self_attention, NEG_INFTY, 0)
    decoder_cross_attention_mask = torch.where(decoder_padding_mask_cross_attention, NEG_INFTY, 0)
    # print f" encoder self _attention mask (encoder_self _attention mask size(l}: (encoder self_attention_maskl, :10, :101")
    # print (f"decoder_self_attention mask {decoder_self_attention_mask.size()}: {decoder_self_attention_mask[0, :10, :10]}")
    # print(f" decoder_cross_attention_mask {decoder_cross_attention_mask.size()}: {decoder_cross_attention_mask[0, :10, : 10]}")
    return encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask

In [38]:
transformer.train()
transformer.to(device)
bst_loss = float("inf") # Initialize the best loss 
wrse_epochs = 0 # Counter for consecutive worse epochs 
max_wrse_epochs = 3 # Number of consecutive worse epochs before stopping

for epoch in range(num_epochs):
    print(f"EPOCH {epoch + 1}----------------------------------------------------------------------------------------")
    epch_loss = 0
    for start_idx in tqdm(range(0, len(english_sentences), chunk_size), desc="Processing Chunks"):
        end_idx = start_idx + chunk_size
        subset = TextDataset(
            english_sentences[start_idx:end_idx],
            hindi_sentences[start_idx:end_idx]
        )
        train_loader = DataLoader(subset, batch_size=batch_size, shuffle=True)  # Small batch size
        iterator = iter(train_loader)
        for batch_num, batch in enumerate(iterator):
            transformer.train()
            eng_batch, hn_batch = batch
            encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_batch, hn_batch)
            optim.zero_grad()
            hn_predictions = transformer(eng_batch,
                                         hn_batch,
                                         encoder_self_attention_mask.to(device), 
                                         decoder_self_attention_mask.to(device), 
                                         decoder_cross_attention_mask.to(device),
                                         enc_start_token=False,
                                         enc_end_token=False,
                                         dec_start_token=True,
                                         dec_end_token=True)
            labels = transformer.decoder.sentence_embedding.batch_tokenize(hn_batch, start_token=False, end_token=True)
            loss = criterian(
                hn_predictions.view(-1, hn_vocab_size).to(device),
                labels.view(-1).to(device)
            ).to(device)
            valid_indicies = torch.where(labels.view(-1) == hindi_to_index[PADDING_TOKEN], False, True)
            loss = loss.sum() / valid_indicies.sum()
            loss.backward()
            optim.step()
            epch_loss += loss.item() # Accumulate batch losses
            
            if batch_num % 100 == 0:
                print(f"Iteration {batch_num} : {loss.item()}")
                print(f"English: {eng_batch[0]}")
                print(f"hindi Translation: {hn_batch[0]}")
                hn_sentence_predicted = torch.argmax(hn_predictions[0], axis=1)
                predicted_sentence = ""
                for idx in hn_sentence_predicted:
                    if idx == hindi_to_index[END_TOKEN]:
                        break
                    predicted_sentence += index_to_hindi[idx.item()] + " " # Add a space after each word
                print(f"hindi Prediction: {predicted_sentence}")
    
    
                transformer.eval()
                hn_sentence = ("",)
                eng_sentence = ("put folder in the bin",)
                for word_counter in range(max_sequence_length):
                    encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask= create_masks(eng_sentence, hn_sentence)
                    predictions = transformer(eng_sentence,
                                              hn_sentence,
                                              encoder_self_attention_mask.to(device), 
                                              decoder_self_attention_mask.to(device), 
                                              decoder_cross_attention_mask.to(device),
                                              enc_start_token=False,
                                              enc_end_token=False,
                                              dec_start_token=True,
                                              dec_end_token=False)
                    next_token_prob_distribution = predictions[0][word_counter] # not actual probs
                    next_token_index = torch.argmax(next_token_prob_distribution).item()
                    next_token = index_to_hindi[next_token_index]
                    hn_sentence = (hn_sentence[0] + next_token + " ", ) # Add a space after each word
                    if next_token == END_TOKEN:
                      break
                print(f"Evaluation translation (put folder in the bin) : {hn_sentence}")
                print("-------------------------------------------")
    
    # Evaluate and save model if the epoch improves the best loss 
    print("------------------------------------------------------------------------------------------------------")
    print(f"Epoch {epoch + 1} Loss: {epch_loss}") 
    if epch_loss < bst_loss: 
        bst_loss = epch_loss 
        wrse_epochs = 0 # Reset counter for worse epochs 
        torch.save(transformer, model_path)
        print(f"Saved Best Model at Epoch {epoch + 1} with Loss: {bst_loss}") 
    else: 
        wrse_epochs += 1 # Increment counter for worse epochs 
        print(f"Epoch {epoch + 1} did not improve. Consecutive worse epochs: {wrse_epochs}") 
        
    # Stop training if the loss doesn't improve for 3 consecutive epochs    
    if wrse_epochs >= max_wrse_epochs:
        print(f"Stopping training after {max_wrse_epochs} consecutive worse epochs.") 
        break

EPOCH 1----------------------------------------------------------------------------------------


Processing Chunks:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration 0 : 11.136124610900879
English: when this has been done the bill is ready to be brought before the house
hindi Translation: इस समूची प्रक्रिया के पश्चात विधेयक सदन के समक्ष लाए जाने के लिए तैयार हो जात है
hindi Prediction: कवितावली जिलो लेआऊँ कुटीरोद्योगी पकड़वाले बहुराष्ट्रीय सुविधा बहुराष्ट्रीय जयवंत जिलो कवितावली सुदारों फसलं परपोतों मुर्गीपालन सुदारों मुर्गीपालन जयवंत कवितावली जिलो मुर्गीपालन दुनपुर जिलो सुदारों कुटीरोद्योगी पैंगुइन्स मुर्गीपालन बहुराष्ट्रीय जिलो बहुराष्ट्रीय मुर्गीपालन कवितावली बृजभाषा मुर्गीपालन जिलो कवितावली अंतर्हित वेश्यावृत ममेरा पकड़वाले मुर्गीपालन जयवंत मुर्गीपालन छेड पोलीनियस मुर्गीपालन कवितावली सुदारों सुदारों छेड पकड़वाले पकड़वाले फार्मूले वेश्यावृत ज्यूडिशियल सुदारों कवितावली किरासन फसलं कवितावली कवितावली लेआऊँ गुफाएं बहुराष्ट्रीय सुदारों मुर्गीपालन कवितावली लेआऊँ कवितावली बृजभाषा पकड़वाले जिलो उत्कंठा बहुराष्ट्रीय जिलो पुण्यप्रद जिलो पैंगुइन्स कवितावली पकड़वाले दैतिक जिलो कवितावली बहुराष्ट्रीय मूवियों पकड़वाले छेड सुविधाविहीन पैंगुइन्स जिलो 


Processing Chunks:  20%|██        | 1/5 [04:53<19:33, 293.30s/it]

Iteration 0 : 7.438241481781006
English: after pharakka dam was built there was help in the life of fishes in the ganga river
hindi Translation: फरक्का बांध बन जाने से गंगा नदी में हिल्सा मछली के बीजोत्पादन में सहायता मिली है।
hindi Prediction: के के के के के के के के के के के के के के के 
Evaluation translation (put folder in the bin) : ('<END> ',)
-------------------------------------------
Iteration 100 : 7.4728851318359375
English: and when an officer suffered bullet injuries in the stomach the dgp philosophised pet bara zalim hota hai goli bhi pacha jayega lrb the stomach is cruel
hindi Translation: और जब एक अधिकारी के पेट में गोली लगी तो ड़ीजीपी ने दार्शनिक अंदाज में कहा       पेट बड़ जालिम होता है   गोली भी पचा जाएगा
hindi Prediction: के के के के के के के के के के के के के के 
Evaluation translation (put folder in the bin) : ('के के <END> ',)
-------------------------------------------
Iteration 200 : 7.182496547698975
English: future reviews
hindi Translation: भावी समीक्षाएं
hi

...

Iteration 0 : 4.65474796295166
English: poor administration
hindi Translation: खराब प्रशासन
hindi Prediction: भारत करने के 
Evaluation translation (put folder in the bin) : ('एक साल के लिए <END> ',)
-------------------------------------------
Iteration 100 : 4.850268363952637
English: watch kung fu movies
hindi Translation: कुंग फू फिल्मे देखा करता था
hindi Prediction: एक एं 
Evaluation translation (put folder in the bin) : ('एक साल के लिए <END> ',)
-------------------------------------------
Iteration 200 : 4.842039585113525
English: and 266 its implies the money which is come to the assembly and for it works is called country fund this is under work of the assembly and this fund is suspect on high level and imposed the states country fund also
hindi Translation: अनु     इसका वर्णन भी करता है वह धन जिसे भारत सरकार कर एकत्रीकरण प्राप्त आय उगाहे गये ऋण के अलावा एकत्र करे भारत की लोकनिधि कहलाती है कर्मचारी भविष्य निधि को भारत की लोकनिधि मे जमा किया गया है यह कार्यपालिका के अधीन निधि है इ

Processing Chunks:  80%|████████  | 4/5 [23:37<06:15, 375.75s/it]

Iteration 0 : 4.900001525878906
English: election commission s procedure function
hindi Translation: निर्वाचन आयोग की कार्यप्रणाली कार्य
hindi Prediction: राष्ट्रपति बिल के नियुक्ति के 
Evaluation translation (put folder in the bin) : ('एक एक साल में <END> ',)
-------------------------------------------
Iteration 100 : 4.5956573486328125
English: what were the gifts to be given to them in case they show friendly feeling
hindi Translation: यदि उन्होंने मित्रता का व्यवहार किया तब उन्हें क्या उपहार दिए जायें
hindi Prediction: वे वे इस को काम किया तो वह एक किया से जाने 
Evaluation translation (put folder in the bin) : ('एक साल के लिए <END> ',)
-------------------------------------------
Iteration 200 : 4.858831882476807
English: even according to the present congress policy it is our objective to nationalise key industries and key services and banking etc
hindi Translation: कांग्रेस की मौजूदा नीति के मुताबिक हमारा उद्देश्य बुनियादी उद्योगों   बुनियादी सेवाओं   बैंकों आदि का राष्ट्रीयकरण कर

Processing Chunks: 100%|██████████| 5/5 [28:05<00:00, 337.07s/it]


Epoch 10 Loss: 18983.889060020447
Saved Best Model at Epoch 10 with Loss: 18983.889060020447


## Load the model

In [47]:
from torch.serialization import add_safe_globals

# Add the Transformer class to the allowlist for loading
add_safe_globals([Transformer])

# Load the model
transformer_model = torch.load(model_path, weights_only=False)

In [48]:
def translate(eng_sentence):
    eng_sentence = (eng_sentence, )
    hn_sentence = ("",)
    for word_counter in range(max_sequence_length):
        encoder_self_attention_mask, decoder_self_attention_mask, decoder_cross_attention_mask = create_masks(eng_sentence, hn_sentence)
        predictions = transformer_model(eng_sentence,
                                          hn_sentence, 
                                          encoder_self_attention_mask.to(device), 
                                          decoder_self_attention_mask.to(device),
                                          decoder_cross_attention_mask.to(device), 
                                          enc_end_token=False, 
                                          dec_start_token=True, 
                                          dec_end_token=False)
        
        next_token_prob_distribution = predictions[0][word_counter]
        next_token_index = torch.argmax(next_token_prob_distribution).item()
        next_token = index_to_hindi[next_token_index]
        hn_sentence = (hn_sentence[0] + next_token + " ", )
        if next_token == END_TOKEN:
            break
    return hn_sentence[0]
    

In [49]:
translation = translate("India is a country")
print(translation)

यह है कि भारत में <END> 
